In [116]:
import pandas as pd
import spacy 
import nltk
nltk.download('punkt')
nlp = spacy.load('ru_core_news_sm')
%pip install natasha
import natasha

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\misha\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.3 -> 23.3.2
[notice] To update, run: C:\Users\misha\AppData\Local\Microsoft\WindowsApps\PythonSoftwareFoundation.Python.3.11_qbz5n2kfra8p0\python.exe -m pip install --upgrade pip


In [117]:
namesTeachers = pd.read_csv('../data/namesTeachers.csv')
namesStudents = pd.read_csv('../data/namesStudents.csv')
namesCombined = pd.read_csv('../data/namesCombined.csv')

In [118]:
names = pd.concat([namesStudents, namesTeachers, namesCombined], ignore_index=True)
names.to_csv('names.csv', index=False)

In [119]:
names['Meaning'].dtype

dtype('O')

In [120]:
names['Meaning'] = names['Meaning'].str.lower()
names.index +=1

In [121]:
names.head(10)

,Names,Meaning
1,Мукам,мелодия
2,Юлия,учтивая
3,Полина,маленькая
4,Ангелина,ангельская
5,Александра,защитница
6,Анастасия,воскрешение
7,Медея,мысль
8,Игорь,хладнокровный воин
9,Михаил,подобный богу
10,Анна,польза


In [122]:
surnameTeachers = pd.read_csv('../data/Surnames_teachers.csv')
surnameStudents = pd.read_csv('../data/Surnames_students.csv')

In [123]:
surname = pd.concat([surnameTeachers, surnameStudents], ignore_index=False)
surname.head(10)

,surname,meaning,Meaning
0,Вербоватая,аппаратчик азотирования,NaN
1,Годованная,воспитанник,NaN
2,Евтушенко,счастливый,NaN
3,Егорова,земледелец,NaN
4,Кривкова,кривой,NaN
5,Кужелева,кожа,NaN
6,Матвеева,божий дар,NaN
7,Меликян,владыка,NaN
8,Михеева,богоподобный,NaN
9,Морозова,мороз,NaN


In [124]:
surname['Meaning'] = surname['meaning']
surname = surname.drop('meaning', axis=1)
surname.index +=1
surname.head(10)

,surname,Meaning
1,Вербоватая,аппаратчик азотирования
2,Годованная,воспитанник
3,Евтушенко,счастливый
4,Егорова,земледелец
5,Кривкова,кривой
6,Кужелева,кожа
7,Матвеева,божий дар
8,Меликян,владыка
9,Михеева,богоподобный
10,Морозова,мороз


In [125]:
surname['Meaning'].dropna(inplace=True)
surname['Meaning'] = surname['Meaning'].astype(str)
surname.to_csv('surname.csv', index=False)

In [126]:
from gensim.models import KeyedVectors
from gensim.models import Word2Vec

# Load the pretrained model from RusVectores
pretrained_model_path = "../model/model.bin"
modelName = KeyedVectors.load_word2vec_format(pretrained_model_path, binary=True)
modelSurnames = Word2Vec.load("../test/surname_word2vec.model")

In [127]:
modelName.most_similar('церковь_PROPN')

[('церковь_NOUN', 0.7719317078590393),
 ('православный_ADJ', 0.6665977835655212),
 ('церковь::христов_PROPN', 0.6490603089332581),
 ('церкоовь_NOUN', 0.6288841962814331),
 ('церкви_NOUN', 0.6126194000244141),
 ('собор_PROPN', 0.6010475158691406),
 ('церковью_NOUN', 0.5924161076545715),
 ('церковный_ADJ', 0.5912197232246399),
 ('паства_NOUN', 0.5878304243087769),
 ('церквь_NOUN', 0.5868114233016968)]

In [128]:
import spacy

# Загрузка модели языка
nlp = spacy.load("ru_core_news_sm")

# Функция для извлечения корневого слова из предложения
def extract_root_word(sentence):
    doc = nlp(sentence)
    for token in doc:
        if token.dep_ == 'ROOT':
            return token.lemma_
    return ''  # Возвращаем пустую строку, если не удалось извлечь корневое слово

# Обновление столбца "Meaning" с корневыми словами
names['Meaning'] = names['Meaning'].apply(extract_root_word)
surname['Meaning'] = surname['Meaning'].apply(extract_root_word)


In [129]:
from nltk.tokenize import sent_tokenize
# Токенизация каждого элемента в столбце и создание нового списка предложений
names['Meaning'] = [sent_tokenize(meaning, language='russian') for meaning in names['Meaning']]
surname['Meaning'] = [sent_tokenize(meaning, language='russian') for meaning in surname['Meaning']]

In [130]:
names['Meaning'] = names['Meaning'].apply(lambda tokens: ' '.join(tokens))
surname['Meaning'] = surname['Meaning'].apply(lambda tokens: ' '.join(tokens))

In [131]:
def lemmatize_with_pos(text, nlp_model):
    doc = nlp_model(text)
    lemmas_with_pos = []
    for token in doc:
        lemmas_with_pos.append(f"{token.lemma_}_{token.pos_}")

    return lemmas_with_pos

names["Meaning"] = names["Meaning"].apply(lambda text: lemmatize_with_pos(text, nlp))


In [132]:
names['Embeddings'] = names['Meaning'].apply(lambda tokens: [modelName[token] if token in modelName else np.zeros(modelName.vector_size) for token in tokens])

In [133]:
from gensim.models import Word2Vec
import numpy as np

def vectorize_meanings(data, model):
    vectorized_meanings = []
    # Загрузка ранее сохраненной модели Word2Vec
    # Векторизация столбца "Tokenized_Meaning"
    for tokens in data["Meaning"]:
        vectors = [model.wv[token] for token in tokens if token in model.wv]
        mean_vector = np.mean(vectors, axis=0) if vectors else np.zeros(model.vector_size)
        vectorized_meanings.append(mean_vector)
    # Добавление векторизованных значений обратно в DataFrame
    data["Vectors"] = vectorized_meanings
    return data

surnames = vectorize_meanings(surname, modelSurnames)



In [134]:
names['Embeddings'] = names['Embeddings'].apply(lambda x: np.array(x, dtype=float))
surname['Vectors'] = surname['Vectors'].apply(lambda x: np.array(x, dtype=float))

In [135]:
import numpy as np
def k_means(data, k):
    # Инициализация центроид
    centroids = data[np.random.choice(range(len(data)), size=k)]

    while True:
        # Выделение кластеров
        clusters = [[] for i in range(k)]
        for point in data:
            distances = [np.linalg.norm(point - centroid) for centroid in centroids]
            closest_centroid_idx = np.argmin(distances)
            clusters[closest_centroid_idx].append(point)

        # Обновление центроид
        new_centroids = []
        for cluster in clusters:
            if len(cluster) > 0:
                cluster_mean = np.mean(cluster, axis=0)
                new_centroids.append(cluster_mean)

        # Проверка на сходимость
        if np.allclose(centroids, new_centroids):
            break

        centroids = new_centroids

    return centroids

k = 1
namesArray = np.array(names['Embeddings'].tolist(), dtype=float)
surnamesArray = np.array(surname['Vectors'].tolist(), dtype=float)
centroidName = k_means(namesArray, k)
centroidSurname = k_means(surnamesArray, k)
print("Центроида:", centroidName)
print("Центроида:", centroidSurname)


Центроида: [array([[ 2.37020178e-02, -6.89571204e-02,  1.43504778e-01,
        -1.12485672e-01, -8.38159226e-03, -1.04834963e-02,
        -9.29379039e-02, -5.53149169e-02,  1.00229690e-01,
        -1.25316251e-01,  2.04728200e-01, -1.04452252e-01,
        -1.36675965e-01, -1.82465001e-02, -6.28367449e-02,
         1.07879126e-02,  1.82309874e-02,  3.06408619e-02,
        -5.38044136e-02,  9.78390725e-02, -1.47177858e-01,
         1.84053887e-03, -6.10881362e-02, -2.04934027e-02,
         1.36864898e-01,  1.30721211e-01, -5.48509097e-02,
        -1.87324215e-01,  2.07202270e-02, -2.12198345e-01,
         4.70057024e-02,  1.34688953e-01, -4.85613759e-02,
        -7.27612360e-02, -6.10350184e-02, -1.09689051e-01,
         1.15040083e-01,  7.13605094e-02,  1.69565500e-01,
        -1.31510114e-01,  4.33182632e-02, -1.93911237e-01,
         2.53730630e-02,  1.44123395e-01,  1.02279683e-01,
        -1.30476947e-01,  1.06469177e-01, -5.19536337e-02,
         5.24802158e-02,  6.38384412e-02, -5

In [136]:
surname.head(10)

,surname,Meaning,Vectors
1,Вербоватая,аппаратчик,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
2,Годованная,воспитанник,"[-0.008358482271432877, 0.010207701474428177, ..."
3,Евтушенко,счастливый,"[-0.008358482271432877, 0.010207701474428177, ..."
4,Егорова,земледелец,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
5,Кривкова,кривой,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
6,Кужелева,кожа,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
7,Матвеева,дар,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
8,Меликян,владыка,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
9,Михеева,богоподобный,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
10,Морозова,мороз,"[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."


In [137]:
def find_closest_vector(centroid, vectors):
    min_distance = float('inf')
    closest_vector = None
    
    for vector in vectors:
        distance = np.linalg.norm(centroid - vector)
        if distance < min_distance:
            min_distance = distance
            closest_vector = vector
    
    return closest_vector

embNames = names['Embeddings']
embSurnames = surname['Vectors']
for vector in embNames:
    closest_vector1 = find_closest_vector(vector, centroidName)
for vector in embSurnames:
    closest_vector2 = find_closest_vector(vector, centroidSurname)
print("Ближайщий вектор имена: ", closest_vector1)
print("Ближайший вектор фамилии: ", closest_vector2)

Ближайщий вектор имена:  [[ 2.37020178e-02 -6.89571204e-02  1.43504778e-01 -1.12485672e-01
  -8.38159226e-03 -1.04834963e-02 -9.29379039e-02 -5.53149169e-02
   1.00229690e-01 -1.25316251e-01  2.04728200e-01 -1.04452252e-01
  -1.36675965e-01 -1.82465001e-02 -6.28367449e-02  1.07879126e-02
   1.82309874e-02  3.06408619e-02 -5.38044136e-02  9.78390725e-02
  -1.47177858e-01  1.84053887e-03 -6.10881362e-02 -2.04934027e-02
   1.36864898e-01  1.30721211e-01 -5.48509097e-02 -1.87324215e-01
   2.07202270e-02 -2.12198345e-01  4.70057024e-02  1.34688953e-01
  -4.85613759e-02 -7.27612360e-02 -6.10350184e-02 -1.09689051e-01
   1.15040083e-01  7.13605094e-02  1.69565500e-01 -1.31510114e-01
   4.33182632e-02 -1.93911237e-01  2.53730630e-02  1.44123395e-01
   1.02279683e-01 -1.30476947e-01  1.06469177e-01 -5.19536337e-02
   5.24802158e-02  6.38384412e-02 -5.98811963e-02 -9.51463459e-02
  -1.15455884e-01  5.72023279e-02  6.30839764e-02  6.02559335e-02
  -1.51960938e-01  2.08921729e-02  1.65623843e-01  

In [138]:
closest_vector1 = closest_vector1.reshape(300,)
closest_vector2 = closest_vector2.reshape(100,)

In [145]:
from gensim.models import Word2Vec

decoded_word_name = modelName.most_similar(positive=[closest_vector1], topn=2)
decoded_word_surname = modelSurnames.wv.most_similar(closest_vector2, topn=3)
print((decoded_word_name[1][0]).replace("_ADJ", "") + ' ' + decoded_word_surname[2][0])


всепрощающий Бог
